In [ ]:
from zipfile import ZipFile
file="/content/drive/My Drive/Project_Data_Arpit/25_image_data.zip"
with ZipFile(file, "r") as zip:
  zip.extractall()
  print("done")

In [ ]:
pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
import matplotlib
matplotlib.use("Agg")
import numpy as np
import pickle
import cv2
import os
import dlib
from keras import backend as K
from keras_vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Dropout, Input, AveragePooling2D
from keras.models import Model, load_model
from keras.activations import relu
from keras.optimizers import RMSprop, SGD, Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import np_utils
from imutils import paths
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
dataset="/content/25_image_data"
le_="/content/drive/My Drive/Project_Data_Arpit/test_data/low_data_model/25_people_le.pickle"
model1 = "/content/drive/My Drive/Project_Data_Arpit/test_data/low_data_model/25_people.model"
plot= "/content/drive/My Drive/Project_Data_Arpit/test_data/low_data_model/25_people_model.png"

In [ ]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
learning_rate = 1e-4
BS = 10
EPOCHS = 30

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(dataset))
data = []
labels = []
count = []

for imagePath in imagePaths:
# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]
	count.append(label)
	print("[INFO]{} - {}/7500 Photos Loaded!".format(label,len(count)))
	image = cv2.imread(imagePath)
	image = image.astype("float")
	image = preprocess_input(image)
	image = cv2.resize(image, (150,150))
	data.append(image)
	labels.append(label)

# convert the data into a NumPy array, then preprocess it by scaling
# all pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0

# encode the labels (which are currently strings) as integers and then
# one-hot encode them
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = np_utils.to_categorical(labels,len(le.classes_))

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, 
                         zoom_range=0.15,
												 width_shift_range=0.2,
												 height_shift_range=0.2,
												 shear_range=0.15,
												 horizontal_flip=True,
												 fill_mode="nearest")

# initialize the optimizer and model
print("[INFO] compiling model...")
opt = Adam(lr = learning_rate, decay= learning_rate / EPOCHS)

#Loading the VggFace default VGG16 model
model = VGGFace(model='vgg16', weights='vggface', include_top=False, input_shape=(150,150,3))

# don't train existing weights
for layer in model.layers:
  layer.trainable = False

# our classification layers
x = Flatten()(model.output)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(len(le.classes_), activation="softmax")(x)

# create a model object
model = Model(inputs=model.input, outputs=predictions)
model.summary()

# Compliling the model with the optimizer 
model.compile(loss="categorical_crossentropy", optimizer = opt, 
              metrics=["accuracy"])

# train the network
print("[INFO] training network for {} epochs...".format(EPOCHS))
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),
											 validation_data=(testX, testY), 
											 steps_per_epoch= len(trainX)//BS, 
											 epochs=EPOCHS)

# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=BS)
print(classification_report(testY.argmax(axis=1),
														predictions.argmax(axis=1),
														target_names=le.classes_))

#save the network to disk
print("[INFO] serializing network to '{}'...".format(model1))
model.save(model1)

#save the label encoder to disk
f = open(le_, "wb")
f.write(pickle.dumps(le))
f.close()

#construct a plot that plots and saves the training history
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plot)

[INFO]Shilpvekariya - 4349/7500 Photos Loaded!
[INFO]Shilpvekariya - 4350/7500 Photos Loaded!
[INFO]Shilpvekariya - 4351/7500 Photos Loaded!
[INFO]Shilpvekariya - 4352/7500 Photos Loaded!
[INFO]Shilpvekariya - 4353/7500 Photos Loaded!
[INFO]Shilpvekariya - 4354/7500 Photos Loaded!
[INFO]Shilpvekariya - 4355/7500 Photos Loaded!
[INFO]Shilpvekariya - 4356/7500 Photos Loaded!
[INFO]Shilpvekariya - 4357/7500 Photos Loaded!
[INFO]Shilpvekariya - 4358/7500 Photos Loaded!
[INFO]Shilpvekariya - 4359/7500 Photos Loaded!
[INFO]Shilpvekariya - 4360/7500 Photos Loaded!
[INFO]Shilpvekariya - 4361/7500 Photos Loaded!
[INFO]Shilpvekariya - 4362/7500 Photos Loaded!
[INFO]Shilpvekariya - 4363/7500 Photos Loaded!
[INFO]Shilpvekariya - 4364/7500 Photos Loaded!
[INFO]Shilpvekariya - 4365/7500 Photos Loaded!
[INFO]Shilpvekariya - 4366/7500 Photos Loaded!
[INFO]Shilpvekariya - 4367/7500 Photos Loaded!
[INFO]Shilpvekariya - 4368/7500 Photos Loaded!
[INFO]Shilpvekariya - 4369/7500 Photos Loaded!
[INFO]Shilpve